In [ ]:
from google.colab import files
uploaded = files.upload()



Saving new_data_urls.csv to new_data_urls.csv


In [ ]:
import pandas as pd
df = pd.read_csv('new_data_urls.csv')
df.head()

,url,status
0,0000111servicehelpdesk.godaddysites.com,0
1,000011accesswebform.godaddysites.com,0
2,00003.online,0
3,0009servicedeskowa.godaddysites.com,0
4,000n38p.wcomhost.com,0


In [ ]:
df.rename(columns={'url': 'URL', 'status':'label'}, inplace=True)

In [ ]:
df.head()

,URL,label
0,0000111servicehelpdesk.godaddysites.com,0
1,000011accesswebform.godaddysites.com,0
2,00003.online,0
3,0009servicedeskowa.godaddysites.com,0
4,000n38p.wcomhost.com,0


In [ ]:
df.shape

(822010, 2)

In [ ]:
df_50000=df.tail(50000)

In [ ]:
data=df

In [ ]:
data.shape

(822010, 2)

In [ ]:
df=df_50000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/add

/content/drive/MyDrive/add


In [ ]:
df.shape

(50000, 2)

In [ ]:
df.head()

,URL,label
772010,e-nvios.cl,0
772011,envirobins.com,0
772012,environaquatech.com,0
772013,environmental-helic.000webhostapp.com,0
772014,environmine.com.au,0


In [ ]:
df['label'].value_counts()

,count
label,
0,50000


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from scipy.stats import entropy
import re


# Feature extraction functions

def is_https(url: str) -> int:
    return 1 if urlparse(url).scheme == "https" else 0

def domain_length(url: str) -> int:
    return len(urlparse(url).netloc)

def url_length(url: str) -> int:
    return len(url)

def dot_count_in_url(url: str) -> int:
    return url.count(".")

def no_of_subdomains(url: str) -> int:
    return len(urlparse(url).netloc.split(".")) - 2

def special_char_ratio_in_url(url: str) -> float:
    special_chars = re.findall(r"[!@#$%^&*()_+={}|<>?]", url)
    return len(special_chars) / len(url) if url else 0

def digit_ratio_in_url(url: str) -> float:
    digits = sum(c.isdigit() for c in url)
    return digits / len(url) if url else 0

def uppercase_count_in_url(url: str) -> int:
    return sum(1 for char in url if char.isupper())

def keyword_in_url(url: str, keyword: str) -> int:
    return 1 if keyword.lower() in url.lower() else 0

def calculate_title_length(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    # Safely check if the title exists before calculating its length
    return len(soup.title.string) if soup.title and soup.title.string else 0


def has_submit_button(html: str) -> int:
    return 1 if "<input type=\"submit\"" in html or "<button" in html else 0

def has_password_field(html: str) -> int:
    return 1 if "<input type=\"password\"" in html else 0

def has_hidden_fields(html: str) -> int:
    return 1 if "<input type=\"hidden\"" in html else 0

def has_favicon(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    return 1 if soup.find("link", rel="icon") or soup.find("link", rel="shortcut icon") else 0

def count_css_lines(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    css_tags = soup.find_all("style") + soup.find_all("link", rel="stylesheet")
    return sum(tag.get_text().count("\n") for tag in css_tags)

def count_js_lines(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    js_tags = soup.find_all("script")
    return sum(tag.get_text().count("\n") for tag in js_tags)

def count_images(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    return len(soup.find_all("img"))

def count_iframes(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    return len(soup.find_all("iframe"))

def line_of_code(html: str) -> int:
    return html.count("\n")

def has_external_form_submit(html: str) -> int:
    soup = BeautifulSoup(html, "html.parser")
    forms = soup.find_all("form")
    return any(form.get("action") and urlparse(form["action"]).netloc != "" for form in forms)

def no_of_qmark_in_url(url: str) -> int:
    return url.count("?")

def tld_length(url: str) -> int:
    return len(urlparse(url).hostname.split(".")[-1]) if urlparse(url).hostname else 0

# URL Entropy Logic
def url_entropy(url: str) -> float:
    url_chars = list(url)
    probabilities = [url_chars.count(c) / len(url_chars) for c in set(url_chars)]
    return entropy(probabilities)

# Add URL entropy to the features dictionary
def extract_features(url: str) -> dict:
    try:
        response = requests.get(url, timeout=5)
        html = response.text
    except Exception:
        html = ""

    features = {
        "HasSocialNet": keyword_in_url(url, "social"),
        "HasCopyrightInfo": keyword_in_url(html, "copyright"),
        "HasDescription": keyword_in_url(html, "description"),
        "IsHTTPS": is_https(url),
        "DomainTitleMatchScore": calculate_title_length(html), # Placeholder
        "HasSubmitButton": has_submit_button(html),
        "IsResponsive": keyword_in_url(html, "viewport"),
        "URLTitleMatchScore": calculate_title_length(html), # Placeholder
        "SpacialCharRatioInURL": special_char_ratio_in_url(url),
        "HasHiddenFields": has_hidden_fields(html),
        "HasFavicon": has_favicon(html),
        "URLCharProb": url_entropy(url), # Placeholder for additional probability logic
        "HasTitle": 1 if BeautifulSoup(html, "html.parser").title else 0,
        "DegitRatioInURL": digit_ratio_in_url(url),
        "URL_Entropy": url_entropy(url),
        "Robots": keyword_in_url(html, "robots"),
        "NoOfJS": count_js_lines(html),
        "LetterRatioInURL": len(re.findall(r"[a-zA-Z]", url)) / len(url) if url else 0,
        "Pay": keyword_in_url(url, "pay"),
        "NoOfOtherSpecialCharsInURL": len(re.findall(r"[#&%]", url)),
        "NoOfSelfRef": keyword_in_url(html, "self-ref"), # Placeholder
        "DomainLength": domain_length(url),
        "NoOfImage": count_images(html),
        "LineOfCode": line_of_code(html),
        "NoOfExternalRef": keyword_in_url(html, "external"), # Placeholder
        "NoOfLettersInURL": sum(c.isalpha() for c in url),
        "URLLength": url_length(url),
        "NoOfiFrame": count_iframes(html),
        "Bank": keyword_in_url(url, "bank"),
        "NoOfDegitsInURL": sum(c.isdigit() for c in url),
        "NoOfQMarkInURL": no_of_qmark_in_url(url),
        "TitleLength": calculate_title_length(html),
        "HasExternalFormSubmit": has_external_form_submit(html),
        "Has_login": keyword_in_url(url, "login"),
        "HasPasswordField": has_password_field(html),
        "DotCount": dot_count_in_url(url),
        "NoOfEmptyRef": keyword_in_url(html, "empty"),
        "Crypto": keyword_in_url(url, "crypto"),
        "TLDLength": tld_length(url),
        "NoOfEqualsInURL": url.count("="),
        "NoOfSelfRedirect": keyword_in_url(html, "self-redirect"),
        "Has_account": keyword_in_url(url, "account"),
        "NoOfCSS": count_css_lines(html),
        "Has_update": keyword_in_url(url, "update"),
        "IsDomainIP": 1 if re.match(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", urlparse(url).hostname or "") else 0,
        "Has_secure": keyword_in_url(url, "secure"),
        "HasObfuscation": keyword_in_url(html, "obfuscation"),
        "NoOfPopup": keyword_in_url(html, "popup"),
        "NoOfURLRedirect": keyword_in_url(html, "redirect"),
        "Has_verify": keyword_in_url(url, "verify"),
        "LargestLineLength": max((len(line) for line in html.splitlines()), default=0),
        "NoOfAmpersandInURL": url.count("&"),
        "Has_password": keyword_in_url(url, "password"),
        "NoOfSubDomain": no_of_subdomains(url),
        "UppercaseCount": uppercase_count_in_url(url),
    }
    return features

# Apply the feature extraction to each URL in the DataFrame
features_df = df['URL'].apply(lambda x: pd.Series(extract_features(x)))

# Concatenate the original URL column with the extracted features
result_df = pd.concat([df, features_df], axis=1)

# Save to CSV
result_df.to_csv("additional_data50000.csv", index=False)
